In [ ]:
import os
import polars as pl
from deltalake import write_deltalake
import json
import time
import csv
from pathlib import Path
from openpyxl import load_workbook, utils
from collections import defaultdict
from typing import Dict, List, Any

def DetectFileType(filepath: str) -> str:
    ext = Path(filepath).suffix.lower()

    if ext == ".csv":
        return "csv"
    elif ext in [".xlsx", ".xls", ".xlsm", ".xlsb"]:
        return "excel"
    else:
        return "unknown"


import polars as pl
from openpyxl import load_workbook


import polars as pl
from openpyxl import load_workbook


from openpyxl import load_workbook
from typing import Dict, List
import polars as pl


def ReadExcelFile(
    origin_filepath: str,
    skip_rows: int,
    n_rows: int,
    schema_dict: Dict[str, pl.DataType] | None = None,
    base_columns: List[str] | None = None,
) -> pl.DataFrame:
    try:
        wb = load_workbook(origin_filepath, read_only=False, data_only=True)
        sh = wb[wb.sheetnames[0]]

        # ===== MERGED-CELL MAP =====
        merged_map = {}
        for rng in sh.merged_cells. ranges:
            master = sh.cell(rng.min_row, rng.min_col).value
            for r in range(rng.min_row, rng.max_row + 1):
                for c in range(rng.min_col, rng.max_col + 1):
                    merged_map[(r, c)] = master

        # ===== HEADERS =====
        header_row = skip_rows + 1

        raw_headers = []
        for c in range(1, sh.max_column + 1):
            cell = sh.cell(header_row, c)
            val = merged_map.get((cell.row, cell.column), cell.value)
            raw_headers.append(str(val) if val else f"column_{c}")

        # ===== DUPLICATE HANDLING WITH NUMERIC SUFFIX =====
        col_names, cnt = [], {}
        for h in raw_headers:
            if h not in cnt:
                cnt[h] = 0
                col_names.append(h)
            else:
                cnt[h] += 1
                col_names. append(f"{h}_{cnt[h]}")

        # ===== FIND REAL LAST NON-EMPTY ROW =====
        real_last_row = 0
        for r in range(header_row + 1, sh.max_row + 1):
            row_values = [
                merged_map.get((r, c), sh.cell(r, c).value)
                for c in range(1, sh.max_column + 1)
            ]
            if any(v not in (None, "") for v in row_values):
                real_last_row = r
            else:
                break  # first empty row → stop scanning

        if real_last_row == 0:
            return pl.DataFrame()

        # ===== SAFE END ROW =====
        data_start = header_row + 1
        data_end = min(data_start + n_rows - 1, real_last_row)

        # ===== READ DATA WITH MERGED CELL VALUES FILLED =====
        columns = {i: [] for i in range(len(col_names))}

        for row in sh.iter_rows(min_row=data_start, max_row=data_end, values_only=False):
            for col_idx, cell in enumerate(row):
                if col_idx >= len(col_names):
                    break
                # Get value from merged_map if cell is merged, otherwise use cell value
                v = merged_map.get((cell.row, cell.column), cell.value)
                columns[col_idx].append(v)

        wb.close()

        # ===== NORMALIZE COLUMN LENGTHS =====
        max_len = max((len(col) for col in columns.values()), default=0)
        for col in columns. values():
            col.extend([None] * (max_len - len(col)))

        # ===== STRINGIFY AND CREATE DATAFRAME =====
        final_data = {
            col_names[i]: [str(v) if v is not None else None for v in columns[i]]
            for i in columns
        }

        df = pl.DataFrame(final_data, infer_schema_length=0)

        # ===== APPLY SCHEMA IF PROVIDED =====
        if schema_dict:
            for col, dtype in schema_dict. items():
                if col in df.columns:
                    df = df.with_columns(pl.col(col). cast(dtype, strict=False))

        # ===== SELECT BASE COLUMNS IF PROVIDED =====
        if base_columns:
            df = df. select([c for c in base_columns if c in df.columns])

        return df

    except Exception as e:
        print("❌ Error:", e)
        return pl.DataFrame()


# def ExcelReadChunkAndApplySchema(
#     origin_filepath: str,
#     skip_rows: int,
#     n_rows: int,
#     schema_dict: Dict[str, pl.DataType] | None = None,
#     base_columns: List[str] | None = None,
# ) -> pl.DataFrame:
#     try:
#         # ===== 1) Build merged cell map =====
#         workbook = load_workbook(filename=origin_filepath, read_only=False, data_only=True)
#         sheet = workbook[workbook.sheetnames[0]]

#         merged_map = {}
#         for merged_range in sheet.merged_cells.ranges:
#             master_value = sheet.cell(row=merged_range.min_row, column=merged_range.min_col).value
#             for r in range(merged_range.min_row, merged_range.max_row + 1):
#                 for c in range(merged_range.min_col, merged_range.max_col + 1):
#                     merged_map[(r, c)] = master_value

#         # Read header row
#         header_row_num = skip_rows + 1
#         header_row = sheet[header_row_num]
#         column_names = []
#         for col_idx, cell in enumerate(header_row):
#             value = merged_map.get((cell.row, cell.column), cell.value)
#             column_names.append(str(value) if value else f"column_{col_idx + 1}")

#         workbook.close()

#         # ===== 2) Read chunk with merged values =====
#         workbook = load_workbook(filename=origin_filepath, read_only=True, data_only=True)
#         sheet = workbook[workbook.sheetnames[0]]

#         start_row = skip_rows + 1
#         end_row = start_row + n_rows

#         col_data = {}
        
#         for row_idx, row in enumerate(sheet.iter_rows(min_row=start_row, max_row=end_row, values_only=False)):
#             if row_idx >= n_rows:
#                 break
            
#             for col_idx, cell in enumerate(row):
#                 if col_idx not in col_data:
#                     col_data[col_idx] = []
                
#                 value = merged_map.get((cell.row, cell.column), cell.value)
#                 col_data[col_idx].append(value)

#         workbook.close()

#         if not col_data:
#             return pl.DataFrame()

#         # ===== 3) Normalize column lengths =====
#         # Because columns might have different lengths.

#         # Example:
#         # Column 0: 100 rows
#         # Column 1: 98 rows
#         # Column 2: 100 rows

#         # Polars needs all columns to have the same length. This code:
#         # max_rows = max(...) → finds the longest column (100)
#         # col_list.extend([None] * ...) → pads shorter columns with None to reach 100
#         # max_rows = max(len(col) for col in col_data.values())
#         # for col_list in col_data.values():
#         #     col_list.extend([None] * (max_rows - len(col_list)))

#         # ===== 4) Create DataFrame =====
#         data_dict = {column_names[col_idx]: col_data[col_idx] for col_idx in sorted(col_data.keys())}
#         df = pl.DataFrame(data_dict, strict=False)

#         # ===== 5) Apply schema if provided =====
#         if schema_dict:
#             for col, dtype in schema_dict.items():
#                 if col in df.columns:
#                     df = df.with_columns(pl.col(col).cast(dtype, strict=False))

#         # ===== 6) Select base columns if provided =====
#         if base_columns:
#             df = df.select([c for c in base_columns if c in df.columns])

#         return df

#     except Exception as e:
#         print(f"❌ Error reading Excel chunk: {e}")
#         return pl.DataFrame()
    

def MakeParquetChunks(
    newly_created_data_file_guid: str,
    origin_filepath: str,
    delta_dir_network_server_path: str,
    save_temporary_deltalake_and_copy: bool
) -> str:
    target_mb_per_partition: int = 512
    skip_rows: int = 0
    max_sample_rows: int = 50_000
    buffer_rows: int = 50_000

    # Validate inputs
    if not origin_filepath or not os.path.isfile(origin_filepath):
        raise FileNotFoundError(f"Origin CSV not found: {origin_filepath}")
    
    # Temporary local delta lake directory because delta lake is not being saved in network address
    if save_temporary_deltalake_and_copy == True:
        delta_dir = "C:\\TemporaryDeltaLake\\" + newly_created_data_file_guid
    else:
        delta_dir = delta_dir_network_server_path

    # Directories preparation
    os.makedirs(delta_dir, exist_ok=True)

    # -----------------------------
    # Lazy scan CSV for random sample
    # -----------------------------
    #lf = pl.scan_csv(origin_filepath, ignore_errors=True)
    ## Read some rows while skipping some rows to get a better sample
    Extension = DetectFileType(origin_filepath)

    if Extension == "csv":
        df_sample = pl.read_csv(origin_filepath, skip_rows=skip_rows, n_rows=max_sample_rows, ignore_errors=True)
    elif Extension == "excel":
        df_sample = ReadExcelFile(origin_filepath, skip_rows, max_sample_rows, None, None)

    print("df_sample")
    print(df_sample)

    ## Getting the base schema
    schema_dict = {col: dtype for col, dtype in zip(df_sample.columns, df_sample.dtypes)}
    base_columns = list(schema_dict.keys())
 
    # Estimate partition size
    temp_path = "temp_sample.parquet"
    df_sample.write_parquet(temp_path)
    row_size = os.path.getsize(temp_path) / len(df_sample)
    os.remove(temp_path)
    rows_per_partition = max(int((target_mb_per_partition * 1024 * 1024) / row_size), buffer_rows)
    print(f"Target rows per partition ≈ {rows_per_partition:,} ")
 
    # -----------------------------
    # Streaming CSV read + Delta write
    # -----------------------------
    first_chunk = True
    pos = 0
    chunk_idx = 1
 
    ## Since the file is huge we read in chunks and write directly to Delta
    while True:
        print(f"Reading rows {pos:,} → {pos + rows_per_partition - 1:,} ",time.time())
        if Extension == "csv":
            df_chunk = pl.read_csv(
                origin_filepath,
                skip_rows=pos,
                n_rows=rows_per_partition,
                dtypes=schema_dict,
                ignore_errors=True
            ).select(base_columns)
        elif Extension == "excel":
            df_chunk = ReadExcelFile(
                origin_filepath,
                skip_rows=pos,
                n_rows=rows_per_partition,
                schema_dict=schema_dict,
                base_columns=base_columns
            )

        print("df_chunk")
        print(df_chunk)
 
        if df_chunk.is_empty():
            break
 
        write_deltalake(
            delta_dir,
            df_chunk,
            mode="overwrite" if first_chunk else "append",
            schema_mode="merge"
        )
        first_chunk = False
        print(f"→ Written partition #{chunk_idx} with {len(df_chunk):,} rows")
        pos += len(df_chunk)
        chunk_idx += 1
 
    print("\n==========================================")
    print(" FULLY STREAMING CSV → DELTA COMPLETED 🎉")
    print(" Delta path:", delta_dir)
    print("==========================================\n")
    return delta_dir


def main():
    pl.Config.set_fmt_str_lengths(2000)   # widen column display
    pl.Config.set_tbl_width_chars(2000)   # expand table width
    pl.Config.set_tbl_rows(50000)         # show up to 50k rows
    pl.Config.set_tbl_cols(500) 

    MakeParquetChunks(
        "abc",
        "E:\\Dev\\Oxyzo R&D\\RndGit\\OxyzoDummyTemplateExcel.xlsx",
        "E:\\Dev\\Oxyzo R&D\\RndGit\\SourceDeltaLake",
        False
    )

    # MakeParquetChunks(
    #     "abc",
    #     "E:\\Dev\\Oxyzo R&D\\RndGit\\Excels\\OxyzoDummyTemplateTransformed.csv",
    #     "E:\\Dev\\Oxyzo R&D\\RndGit\\SourceDeltaLake",
    #     False
    # )


if __name__ == "__main__":
    main()

df_sample
shape: (5, 19)
┌─────────┬─────────────┬────────┬──────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬───────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┐
│ Loan ID ┆ Loan No.    ┆ Org ID ┆ Org Name ┆ Sanction Date       ┆ Current max DPD     ┆ Lifetime max DPD    ┆ Lifetime max DPD on ┆ Total Overdue       ┆ Principal Overdue   ┆ EMI Overdue         ┆ POS                 ┆ Current max DPD_1   ┆ Lifetime max DPD_1  ┆ Lifetime max DPD on_1 ┆ Total Overdue_1     ┆ Principal Overdue_1 ┆ EMI Overdue_1       ┆ POS_1               │
│ ---     ┆ ---         ┆ ---    ┆ ---      ┆ ---                 ┆ ---                 ┆ ---                 ┆ ---                 ┆ ---                 ┆ ---                 ┆ ---                 ┆ ---                

Exception: External error: Schema error: Invalid data type for Delta Lake: Null

In [ ]:
from deltalake import DeltaTable
import pyarrow as pa
import pyarrow.dataset as ds
from enum import Enum

class DataType(Enum):
    """Enumeration for data types."""
    Unknown = 0
    Numeric = 1
    DateTime = 2
    Float = 3
    String = 4
    Formula = 5
    Blank = 6
    Boolean = 7
    Error = 8

from deltalake import DeltaTable
import pyarrow as pa
import pyarrow.dataset as ds

def map_pyarrow_type(dtype: pa.DataType) -> int:
    if pa.types.is_integer(dtype) or pa.types.is_decimal(dtype):
        return DataType.Numeric.value
    if pa.types.is_floating(dtype):
        return DataType.Float.value
    if pa.types.is_string(dtype):
        return DataType.String.value
    if pa.types.is_boolean(dtype):
        return DataType.Boolean.value
    if pa.types.is_timestamp(dtype) or pa.types.is_date(dtype):
        return DataType.DateTime.value
    return DataType.Unknown.value


def GetFullDetailsOfDeltaLakeFile(folder_path: str):
    """
    Get comprehensive metadata about a Delta Lake table
    
    Returns:
    {
        "StgDataFileInfo": {
            "RowCount": int,
            "ColumnCount": int
        },
        "StgDataFileColumnInfo": [
            {
                "DataFileColumnName": str,
                "ColumnType": str,
                "HasNullValue": bool,
                "DistinctValueCount": int,
                "NullCount": int
            },
            ...
        ]
    }
    """
    try:
        dt = DeltaTable(folder_path)
        
        # Get schema and dataset
        dataset = dt.to_pyarrow_dataset()
        arrow_schema = dataset.schema

        # Initialize column info
        col_info = {}
        for f in arrow_schema:
            col_info[f.name] = {
                "DataFileColumnName": f.name,
                "ColumnType": map_pyarrow_type(f.type),
                "HasNullValue": False,
                "NullCount": 0,
                "DistinctValueCount": set(),
            }

        # Scan batches
        row_count = 0
        scanner = ds.Scanner.from_dataset(dataset, columns=None, batch_size=50_000)

        for batch in scanner.to_batches():
            row_count += len(batch)
            
            for col_name, array in zip(batch.schema.names, batch.columns):
                null_count = array.null_count
                
                if null_count > 0:
                    col_info[col_name]["HasNullValue"] = True
                    col_info[col_name]["NullCount"] += null_count

                # Get distinct values (excluding nulls)
                try:
                    if len(array) > null_count:
                        non_null_array = array.drop_null()
                        uniques = pc.unique(non_null_array)
                        col_info[col_name]["DistinctValueCount"].update(uniques.to_pylist())
                except Exception:
                    pass

        # Convert distinct value sets to counts
        for col in col_info.values():
            col["DistinctValueCount"] = len(col["DistinctValueCount"])

        # Get Delta table metadata
        dt_metadata = dt.metadata()

        return {
            "StgDataFileInfo": {
                "RowCount": row_count,
                "ColumnCount": len(arrow_schema),
            },
            "StgDataFileColumnInfo": list(col_info.values()),
        }

    except Exception as e:
        print(f"Error reading Delta Lake table: {e}")
        return {
            "StgDataFileInfo": {"RowCount": 0, "ColumnCount": 0},
            "StgDataFileColumnInfo": [],
        }


def main():
    data = GetFullDetailsOfDeltaLakeFile("E:\\Dev\\Oxyzo R&D\\RndGit\\SourceDeltaLake")
    print(data)

if __name__ == "__main__":
    main()
